## ETL_ MOVIES
TRANSFORMACIONES: 
- Algunos campos, como **belongs_to_collection**, **production_companies** y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.<P>

- Los valores nulos de los campos **revenue**, **budget** deben ser rellenados por el número 0.<P>

- Los valores nulos del campo **release date** deben eliminarse.<P>
 
- De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna **release_year** donde extraerán el año de la fecha de estreno. <P>

- Crear la columna con el retorno de inversión, llamada **return** con los campos **revenue** y **budget**, dividiendo estas dos últimas **revenue** / **budget**, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0. <P>

- Eliminar las columnas que no serán utilizadas, **video**,**imdb_id**,**adult**,**original_title**,**poster_path** y **homepage**. <P>

In [8]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import ast

### 0. Carga de los datos

In [9]:
movie_test  = pd.read_csv('datasets/movies_dataset.csv', sep=',',quotechar='"', encoding='utf-8' )

C:\Users\Angelica\AppData\Local\Temp\ipykernel_8792\3618576697.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_test  = pd.read_csv('datasets/movies_dataset.csv', sep=',',quotechar='"', encoding='utf-8' )


In [10]:
print(movie_test.shape)

(45466, 24)


Copia del dataset para aplicar las transformaciones

In [11]:
movie_test_df = movie_test.copy()

In [12]:
movie_test_df.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


### 1. TRANSFORMACION - Eliminación columnas

COLUMNAS:
- video, imdb_id, adult, original_title, poster_path y homepage.

In [13]:
columnas = [ 'video', 'imdb_id', 'adult', 'original_title', 'poster_path' , 'homepage']
movie_test_df = movie_test_df.drop(columnas, axis=1)

Columnas innecesarias para el análisis

In [14]:
columnas =['overview','status','tagline'] 
movie_test_df  =movie_test_df.drop (columnas, axis=1)

### 2. TRANSFORMACION -  VALORES NULOS A '0'

COLUMNAS:
- budget
- revenue

Conversión a enteros y valores nulos seteados con '0'
- pd.to_numeric   : convierte los valores de la columna, maneja errores 'coerce' (lo paso a nan)
- fillna(0)       : valores nan a 0
- astype('Int64') : valores enteros

In [15]:
movie_test_df["budget"]= pd.to_numeric(movie_test_df["budget"], errors='coerce').fillna(0).astype('Int64')
movie_test_df["revenue"]= pd.to_numeric(movie_test_df["revenue"], errors= 'coerce').fillna(0).astype('Int64')

In [16]:
#Se verifica cambios
print(type(movie_test["budget"][0]))
print(type(movie_test_df["budget"][0]))

print(type(movie_test["revenue"][0]))
print(type(movie_test_df["revenue"][0]))

<class 'str'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.int64'>


### 3. TRANSFORMACION -  COLUMNA RETURN

return = revenue/budget (recaudacion/presupuesto)

In [17]:
movie_test_df["return"] = movie_test_df.apply(
                        lambda fila: round( fila['revenue']/fila['budget'], 2) if fila['budget']!= 0  else 0, axis = 1)

### 4. TRANSFORMACION 
* ELIMINACIÓN VALORES NULOS 'release date' <p>
* COLUMNA RELEASE_YEAR

- Pasar de string a formato datetime (AAAA-mm-dd)
- Obtiene la columna release_year, a formato Int
- Se elimina las filas donde haya valores nulos en 'release_date'

In [18]:
movie_test_df["release_date"] = pd.to_datetime(movie_test_df["release_date"], errors= 'coerce', format='%Y-%m-%d')
movie_test_df["release_year"] = movie_test_df["release_date"].dt.year.astype('Int64')

In [19]:
movie_test_df = movie_test_df.dropna(subset= 'release_date')

Comprobacion de eliminación de valores nulos

In [20]:
print(movie_test["release_date"].isnull().sum())
print(movie_test_df["release_date"].isnull().sum())

87
0


In [21]:
print(movie_test_df["release_year"].isnull().sum())

0


Comprobación columna 'release_year'

In [22]:
movie_test_df[['release_date','release_year']].head(5)

,release_date,release_year
0,1995-10-30,1995
1,1995-12-15,1995
2,1995-12-22,1995
3,1995-12-22,1995
4,1995-02-10,1995


### 5. TRANSFORMACION - DESANIDAR CAMPOS

Columnas con campos anidados:
- belongs_to_collection  - dicionario
- genres                 - lista de dicionarios
- production_companies   - lista de dicionario
- production_countries   - lista de dicionario
- spoken_languages       - lista de dicionario

Función para desanidar <p>
**Párametros de entrada**: dataframe y columna a desanidar

1. Se extraer los valores del str
2. Verificar si es lista/diccionario
3. Normalizar columnas
4. Cambiar el nombre de columnas normalizadas
5. Elimina la columna original
6. Concatenar las nuevas columnas al dataframe final

In [23]:

def desanidar_concat(df, col):
    # Convertir cadenas de texto en listas/diccionarios si es necesario
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    # Explode para listas de diccionarios o manejar diccionarios directamente
    if df[col].apply(lambda x: isinstance(x,list)).any(): 
        col_norm = pd.json_normalize(df[col].explode().dropna()).reset_index(drop=True)
    else: # para diccionarios
        col_norm = pd.json_normalize(df[col].dropna()).reset_index(drop=True)
    
    # Rename las nuevas columnas
    col_norm = col_norm.add_suffix(f'_{col}')

    #Elimina columna original, agrega columnas nuevas al dataframe
    #df = pd.concat([df.drop(columns=[col]) , col_norm], axis=1)
    df= df.drop(columns=col).join(col_norm)

    return df

In [24]:
movie_test_df=desanidar_concat(movie_test_df, "belongs_to_collection")
movie_test_df=desanidar_concat(movie_test_df, "genres")
movie_test_df=desanidar_concat(movie_test_df, "production_companies")
movie_test_df=desanidar_concat(movie_test_df, "production_countries")
movie_test_df=desanidar_concat(movie_test_df, "spoken_languages")

#for columna in ['genres', 'production_companies', 'production_countries', 'spoken_languages']:
#    movies = desanidar_concat(movies, columna)

Comparación 

In [25]:
print(movie_test.columns)
print(movie_test_df.columns)

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')
Index(['budget', 'id', 'original_language', 'popularity', 'release_date',
       'revenue', 'runtime', 'title', 'vote_average', 'vote_count', 'return',
       'release_year', 'id_belongs_to_collection',
       'name_belongs_to_collection', 'poster_path_belongs_to_collection',
       'backdrop_path_belongs_to_collection', 'id_genres', 'name_genres',
       'name_production_companies', 'id_production_companies',
       'iso_3166_1_production_countries', 'name_production_countries',
       'iso_639_1_spoken_languages', 'name_spoken_languages'],
      dtype='object')


In [26]:
movie_test_df.head(1)

,budget,id,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,...,poster_path_belongs_to_collection,backdrop_path_belongs_to_collection,id_genres,name_genres,name_production_companies,id_production_companies,iso_3166_1_production_countries,name_production_countries,iso_639_1_spoken_languages,name_spoken_languages
0,30000000,862,en,21.946943,1995-10-30,373554033,81.0,Toy Story,7.7,5415.0,...,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,16,Animation,Pixar Animation Studios,3,US,United States of America,en,English


### EILIMINAR COLUMNAS INNECESARIAS

Se analiza valores nulos en cada columna

In [27]:
nulos = movie_test_df.isnull().sum()
print(nulos )

budget                                     0
id                                         0
original_language                         11
popularity                                 0
release_date                               0
revenue                                    0
runtime                                  246
title                                      0
vote_average                               0
vote_count                                 0
return                                     0
release_year                               0
id_belongs_to_collection               40892
name_belongs_to_collection             40892
poster_path_belongs_to_collection      41435
backdrop_path_belongs_to_collection    42120
id_genres                                  0
name_genres                                0
name_production_companies                  0
id_production_companies                    0
iso_3166_1_production_countries            0
name_production_countries                  0
iso_639_1_

In [28]:
# 'overview','status','tagline'
columnas =['id_belongs_to_collection', 'name_belongs_to_collection',
       'poster_path_belongs_to_collection',
       'backdrop_path_belongs_to_collection', 
       'iso_3166_1_production_countries','iso_639_1_spoken_languages'] 
movie_test_df  =movie_test_df.drop (columnas, axis=1)

In [29]:
nulos = movie_test_df.isnull().sum()
print(nulos )

budget                         0
id                             0
original_language             11
popularity                     0
release_date                   0
revenue                        0
runtime                      246
title                          0
vote_average                   0
vote_count                     0
return                         0
release_year                   0
id_genres                      0
name_genres                    0
name_production_companies      0
id_production_companies        0
name_production_countries      0
name_spoken_languages          0
dtype: int64


Comparación dataset original vs ETL(dataset)

In [30]:
print(movie_test.shape)
print(movie_test_df.shape)

(45466, 24)
(45376, 18)


In [31]:
movie_test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   budget                     45376 non-null  Int64         
 1   id                         45376 non-null  object        
 2   original_language          45365 non-null  object        
 3   popularity                 45376 non-null  object        
 4   release_date               45376 non-null  datetime64[ns]
 5   revenue                    45376 non-null  Int64         
 6   runtime                    45130 non-null  float64       
 7   title                      45376 non-null  object        
 8   vote_average               45376 non-null  float64       
 9   vote_count                 45376 non-null  float64       
 10  return                     45376 non-null  float64       
 11  release_year               45376 non-null  Int64         
 12  id_genres

### CAMBIAR NOMBRE y TIPO DATOS (string => int, float, etc)

In [32]:
movie_test_df.dtypes

budget                                Int64
id                                   object
original_language                    object
popularity                           object
release_date                 datetime64[ns]
revenue                               Int64
runtime                             float64
title                                object
vote_average                        float64
vote_count                          float64
return                              float64
release_year                          Int64
id_genres                             int64
name_genres                          object
name_production_companies            object
id_production_companies               int64
name_production_countries            object
name_spoken_languages                object
dtype: object

Cambiar nombres a columnas:

In [33]:
col_renombrar= {'id':'IdMovie','name_genres': 'genres', 'name_production_companies':'production_companies',
'name_production_countries':'production_countries', 'name_spoken_languages':'spoken_languages' }

movie_test_df  =movie_test_df.rename( columns = col_renombrar)

In [34]:
movie_test_df.columns

Index(['budget', 'IdMovie', 'original_language', 'popularity', 'release_date',
       'revenue', 'runtime', 'title', 'vote_average', 'vote_count', 'return',
       'release_year', 'id_genres', 'genres', 'production_companies',
       'id_production_companies', 'production_countries', 'spoken_languages'],
      dtype='object')

Cambiar tipo de datos
- id    (str => int)

In [36]:
movie_test_df["IdMovie"] = movie_test_df["IdMovie"].astype('Int64')
movie_test_df["popularity"] = movie_test_df["popularity"].astype(float)

In [37]:
print(type(movie_test["popularity"][0]))

<class 'float'>


Comprobación cambios

In [41]:
print(type(movie_test["id"][0]))
print(type(movie_test_df["IdMovie"][0]))

<class 'str'>
<class 'numpy.int64'>


Eliminación/Cambio de valores **NULOS**

In [42]:
nulos1 = movie_test_df.isnull().sum()
print(nulos1 )

budget                       0
IdMovie                      0
original_language           11
popularity                   0
release_date                 0
revenue                      0
runtime                    246
title                        0
vote_average                 0
vote_count                   0
return                       0
release_year                 0
id_genres                    0
genres                       0
production_companies         0
id_production_companies      0
production_countries         0
spoken_languages             0
dtype: int64


In [43]:
# inplace=True: afecta directamente al dataframe
movie_test_df["runtime"].fillna(0,inplace=True)
movie_test_df["original_language"].fillna('desconocido',inplace=True)

In [44]:
nulos2 = movie_test_df.isnull().sum()
print(nulos2 )

budget                     0
IdMovie                    0
original_language          0
popularity                 0
release_date               0
revenue                    0
runtime                    0
title                      0
vote_average               0
vote_count                 0
return                     0
release_year               0
id_genres                  0
genres                     0
production_companies       0
id_production_companies    0
production_countries       0
spoken_languages           0
dtype: int64


In [45]:
movie_test_df["original_language"].head(5)

0    en
1    en
2    en
3    en
4    en
Name: original_language, dtype: object

### VERIFICACIÓN IDENTIFICADORES UNICOS

Cantidad de valores duplicados, luego de aparcer por segunda vez <p>
Identificador único: 'id'

In [ ]:
duplicados= movie_test_df[movie_test_df["IdMovie"].duplicated()]
print(len(duplicados))
duplicados

Eliminación de valores duplicados de la columna 'IdMovie'

In [47]:
# keep='first' se mantiene el id original, y se elimina el duplicado
movie_test_df.drop_duplicates(subset='IdMovie', keep='first', inplace=True)

Se verifica que ya no existen valores duplicados

In [48]:
duplicados2= movie_test_df[movie_test_df["IdMovie"].duplicated()]
print(len(duplicados2))
duplicados2

0


,budget,IdMovie,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,return,release_year,id_genres,genres,production_companies,id_production_companies,production_countries,spoken_languages


### CAMBIOS PARA FUNCIONES API

def cantidad_filmaciones_mes(Mes): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.

Se generan Columnas en Español

In [49]:
movie_test_df["release_date"].head(1)
# tipo de dato: datetime64

0   1995-10-30
Name: release_date, dtype: datetime64[ns]

Diccionario meses en Español

In [50]:
meses = {
    1: 'Enero',
    2:'Febrero',
    3: "Marzo",
    4: "Abril",
    5: "Mayo", 
    6: "Junio",
    7: "Julio",
    8: "Agosto", 
    9: "Septiembre", 
    10: "Octubre", 
    11: "Noviembre", 
    12: "Diciembre"
}
meses

{1: 'Enero',
 2: 'Febrero',
 3: 'Marzo',
 4: 'Abril',
 5: 'Mayo',
 6: 'Junio',
 7: 'Julio',
 8: 'Agosto',
 9: 'Septiembre',
 10: 'Octubre',
 11: 'Noviembre',
 12: 'Diciembre'}

- Se crea columna en el dataset 'release_meses'
- Se mapea y se asigna el valor del diccionario correspondiente

In [51]:
movie_test_df["release_meses"] = movie_test_df["release_date"].dt.month.map(meses)

Se verifica creación y asignación correcta

In [52]:
movie_test_df[["release_date", "release_meses"]].head(10)

,release_date,release_meses
0,1995-10-30,Octubre
1,1995-12-15,Diciembre
2,1995-12-22,Diciembre
3,1995-12-22,Diciembre
4,1995-02-10,Febrero
5,1995-12-15,Diciembre
6,1995-12-15,Diciembre
7,1995-12-22,Diciembre
8,1995-12-22,Diciembre
9,1995-11-16,Noviembre


PASOS
- Obtener número del mes de la columna 'col_mes'
- Si número col_mes == key del dic meses
- Se asigna a un dataframe release_mes: mese[key] 

In [53]:
movie_test_df.head(10)

,budget,IdMovie,original_language,popularity,release_date,revenue,runtime,title,vote_average,vote_count,return,release_year,id_genres,genres,production_companies,id_production_companies,production_countries,spoken_languages,release_meses
0,30000000,862,en,21.946943,1995-10-30,373554033,81.0,Toy Story,7.7,5415.0,12.45,1995,16,Animation,Pixar Animation Studios,3,United States of America,English,Octubre
1,65000000,8844,en,17.015539,1995-12-15,262797249,104.0,Jumanji,6.9,2413.0,4.04,1995,35,Comedy,TriStar Pictures,559,United States of America,English,Diciembre
2,0,15602,en,11.712900,1995-12-22,0,101.0,Grumpier Old Men,6.5,92.0,0.00,1995,10751,Family,Teitler Film,2550,United States of America,Français,Diciembre
3,16000000,31357,en,3.859495,1995-12-22,81452156,127.0,Waiting to Exhale,6.1,34.0,5.09,1995,12,Adventure,Interscope Communications,10201,United States of America,English,Diciembre
4,0,11862,en,8.387519,1995-02-10,76578911,106.0,Father of the Bride Part II,5.7,173.0,0.00,1995,14,Fantasy,Warner Bros.,6194,United States of America,English,Febrero
5,60000000,949,en,17.924927,1995-12-15,187436818,170.0,Heat,7.7,1886.0,3.12,1995,10751,Family,Lancaster Gate,19464,United States of America,English,Diciembre
6,58000000,11860,en,6.677277,1995-12-15,0,127.0,Sabrina,6.2,141.0,0.00,1995,10749,Romance,Twentieth Century Fox Film Corporation,306,Germany,English,Diciembre
7,0,45325,en,2.561161,1995-12-22,0,97.0,Tom and Huck,5.4,45.0,0.00,1995,35,Comedy,Sandollar Productions,5842,United States of America,Español,Diciembre
8,35000000,9091,en,5.231580,1995-12-22,64350171,106.0,Sudden Death,5.5,174.0,1.84,1995,35,Comedy,Touchstone Pictures,9195,United States of America,Français,Diciembre
9,58000000,710,en,14.686036,1995-11-16,352194034,130.0,GoldenEye,6.6,1194.0,6.07,1995,18,Drama,Regency Enterprises,508,United States of America,English,Noviembre


### EXPORTACIÓN DATASET FINAL 

In [54]:
print(movie_test_df.shape)
movie_test_df.info()

(45346, 19)
<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   budget                   45346 non-null  Int64         
 1   IdMovie                  45346 non-null  Int64         
 2   original_language        45346 non-null  object        
 3   popularity               45346 non-null  float64       
 4   release_date             45346 non-null  datetime64[ns]
 5   revenue                  45346 non-null  Int64         
 6   runtime                  45346 non-null  float64       
 7   title                    45346 non-null  object        
 8   vote_average             45346 non-null  float64       
 9   vote_count               45346 non-null  float64       
 10  return                   45346 non-null  float64       
 11  release_year             45346 non-null  Int64         
 12  id_genres                

In [55]:
movie_test_df.to_parquet('datasets/movie_dataset_final.parquet', engine='pyarrow')